In [ ]:
## Sentiment Analysis Using Supervised Learning Approach
Purpose: To use Machine Learning Method, using TFIDF Vectorizer to vectorize the corpus, to predict labels

# from keras.models import Sequential
# from keras.layers import Input, Dense, Embedding, SpatialDropout1D, LSTM
# from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm, tree
# from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier

from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn import preprocessing
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

import datetime
import os
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from keras.preprocessing.text import Tokenizer
import string
import seaborn as sns
import spacy
import unidecode
# import contractions as contract
import re
import wordninja
import collections
import pkg_resources
from spellchecker import SpellChecker 
from symspellpy import SymSpell, Verbosity
from wordcloud import WordCloud

import pickle

# Pickle Methodology
def load_pickle(input_dir):
    f = open(input_dir, "rb")
    df = pickle.load(f)
    return df
    
def save_pickle(df, output_dir):
    df.to_pickle(output_dir)

# import labelled dataset from pre-labelled data in RavenPack
import datetime

batch_id = datetime.date.today().strftime("%y%m%d")
df = pd.read_csv(f'outputs/china_news_sentiments.csv', encoding='latin-1')
df = df[['timestamp_tz', 'cleaned_headline', 'class', 'entity_name']]
df = df[df['cleaned_headline'].notna()]
df.reset_index(inplace=True, drop=True)
df

### Vectorize Textual Data
We convert textual data into numerical feature vectors

# -1 (NEGATIVE)
# 1 (POSITIVE)

le = LabelEncoder()
df['class_label'] = le.fit_transform(df['class'])

X_train, X_test, y_train, y_test = train_test_split(df[['cleaned_headline']], df['class_label'], shuffle=True, test_size=0.30)

### Classification Models

### TfidfVectorizer For Corpus

pipeline = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('logreg', LogisticRegression(solver='lbfgs'))
])

pipeline.fit(X_train.cleaned_headline, y_train)

metrics.accuracy_score(y_test, pipeline.predict(X_test.cleaned_headline))

%%time

parameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1,3)],
    'tfidf__max_df': np.linspace(0.16, 0.21, 6),
    'tfidf__min_df': range(7, 15),
}

clf = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=5)
clf.fit(X_train.cleaned_headline, y_train)

clf.best_params_

#### Tune with an updated set of parameter space
(To provide a more conclusive results)

parameters_updated = {
    'tfidf__ngram_range': [(1, 3), (1, 4), (1, 5)],
    'tfidf__max_df': np.linspace(0.10, 0.24, 6),
    'tfidf__min_df': range(1, 10),
}

clf_updated = GridSearchCV(pipeline, parameters_updated, n_jobs=-1, verbose=5)
clf_updated.fit(X_train.cleaned_headline, y_train)

clf_updated.best_params_

#### Use the parameters to vectorize the Corpus column

tfidf_vect = TfidfVectorizer(max_df=0.1, min_df=3, ngram_range=(1, 3)) # TfidfVectorizer(max_df=0.16, min_df=5, ngram_range=(1, 3))
X_train_dtm = tfidf_vect.fit_transform(X_train.cleaned_headline)
X_test_dtm = tfidf_vect.transform(X_test.cleaned_headline)

### XGBoost Model

%%time

xg_params = {'max_depth': range(3,10,1), 
             'min_child_weight': range(1,8,2)
            }

xg = XGBClassifier()

xg_clf = GridSearchCV(xg, xg_params, verbose=5, n_jobs=-1)
xg_clf.fit(X_train_dtm, y_train)

metrics.accuracy_score(y_test, xg_clf.predict(X_test_dtm))

xg_clf.best_params_

%%time

xg_params_updated = {'max_depth': range(1,11,2), 
             'min_child_weight': range(1,21,2)
            }

xg = XGBClassifier()

xg_clf_updated = GridSearchCV(xg, xg_params_updated, verbose=5, n_jobs=-1)
xg_clf_updated.fit(X_train_dtm, y_train)

xg_clf_updated.best_params_

xg = XGBClassifier(max_depth=7, min_child_weight=1)

xg.fit(X_train_dtm, y_train)

metrics.accuracy_score(y_test, xg.predict(X_test_dtm))

target_names = ['Negative', 'Positive']

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, xg.predict(X_test_dtm)))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, xg.predict(X_test_dtm), digits=3, target_names=target_names))

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

classes = np.unique(y_test)
fig, ax = plt.subplots()
cm = metrics.confusion_matrix(y_test, xg.predict(X_test_dtm), labels=classes)
sns.heatmap(cm, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False)
ax.set(xlabel="Pred", ylabel="True", title="Confusion matrix")
ax.set_yticklabels(labels=classes, rotation=0)
plt.show()

# roc curve and auc

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
    
probs = xg.predict_proba(X_test_dtm)
probs = probs[:, 1]
auc = roc_auc_score(y_test, probs)
print('AUC: %.2f' % auc)
fpr, tpr, thresholds = roc_curve(y_test, probs)
plot_roc_curve(fpr, tpr)

### SVM
Test both Radial and Linear Basis for SVM model training
Reference: https://www.researchgate.net/publication/305623869_Sentiment_Classification_of_Financial_News_Using_Statistical_Features

%%time

svm_params = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01]}

sv = svm.SVC()

svm_clf = GridSearchCV(sv, svm_params, verbose=5, n_jobs=-1)
svm_clf.fit(X_train_dtm, y_train)

svm_clf.best_params_

metrics.accuracy_score(y_test, svm_clf.predict(X_test_dtm))

svm_params = {'C': [0.1, 1, 10, 100, 1000, 10000], 'gamma': [1000, 100, 10, 1, 0.1, 0.01]}

sv = svm.SVC()

svm_clf = GridSearchCV(sv, svm_params, verbose=5, n_jobs=-1)
svm_clf.fit(X_train_dtm, y_train)
svm_clf.best_params_

sv = svm.SVC(C=10, gamma=0.1)

sv.fit(X_train_dtm, y_train)

metrics.accuracy_score(y_test, sv.predict(X_test_dtm))

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, sv.predict(X_test_dtm)))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, sv.predict(X_test_dtm), digits=3, target_names=target_names))

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

classes = np.unique(y_test)
fig, ax = plt.subplots()
cm = metrics.confusion_matrix(y_test, sv.predict(X_test_dtm), labels=classes)
sns.heatmap(cm, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False)
ax.set(xlabel="Pred", ylabel="True", title="Confusion matrix")
ax.set_yticklabels(labels=classes, rotation=0)
plt.show()

# roc curve and auc
sv = svm.SVC(C=100, gamma=0.01, probability=True) # made probability=True
sv.fit(X_train_dtm, y_train)

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
    
probs = sv.predict_proba(X_test_dtm)
probs = probs[:, 1]
auc = roc_auc_score(y_test, probs)
print('AUC: %.2f' % auc)
fpr, tpr, thresholds = roc_curve(y_test, probs)
plot_roc_curve(fpr, tpr)

### Logistic Regression

lr = LogisticRegression()

lr.fit(X_train_dtm, y_train)

metrics.accuracy_score(y_test, lr.predict(X_test_dtm))

parameters_logreg = {
    'penalty': ['l1', 'l2'],
    'C': [100, 10, 1.0, 0.1, 0.01]
}

lr = LogisticRegression()
lr_clf = GridSearchCV(lr, parameters_logreg, verbose=5, n_jobs=-1)
lr_clf.fit(X_train_dtm, y_train)
lr_clf.best_params_

lr = LogisticRegression(C=10, penalty='l2')

lr.fit(X_train_dtm, y_train)

metrics.accuracy_score(y_test, lr.predict(X_test_dtm))

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

classes = np.unique(y_test)
fig, ax = plt.subplots()
cm = metrics.confusion_matrix(y_test, lr.predict(X_test_dtm), labels=classes)
sns.heatmap(cm, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False)
ax.set(xlabel="Pred", ylabel="True", title="Confusion matrix")
ax.set_yticklabels(labels=classes, rotation=0)
plt.show()

# roc curve and auc

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
    
probs = lr.predict_proba(X_test_dtm)
probs = probs[:, 1]
auc = roc_auc_score(y_test, probs)
print('AUC: %.2f' % auc)
fpr, tpr, thresholds = roc_curve(y_test, probs)
plot_roc_curve(fpr, tpr)

### KNN Model

%%time

knn_params = {'n_neighbors': np.arange(1,100,2)}

knn = KNeighborsClassifier()

knn_clf = GridSearchCV(knn, knn_params, verbose=5, n_jobs=-1)
knn_clf.fit(X_train_dtm, y_train)
knn_clf.best_params_

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train_dtm, y_train)

metrics.accuracy_score(y_test, knn.predict(X_test_dtm))

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, knn.predict(X_test_dtm)))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, knn.predict(X_test_dtm), digits=3, target_names=target_names))

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

classes = np.unique(y_test)
fig, ax = plt.subplots()
cm = metrics.confusion_matrix(y_test, knn.predict(X_test_dtm), labels=classes)
sns.heatmap(cm, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False)
ax.set(xlabel="Pred", ylabel="True", title="Confusion matrix")
ax.set_yticklabels(labels=classes, rotation=0)
plt.show()

# roc curve and auc

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
    
probs = knn.predict_proba(X_test_dtm)
probs = probs[:, 1]
auc = roc_auc_score(y_test, probs)
print('AUC: %.2f' % auc)
fpr, tpr, thresholds = roc_curve(y_test, probs)
plot_roc_curve(fpr, tpr)

### Naive Bayes Model
Commonly Used Model:<br>
https://itechindia.co/blog/which-of-the-3-algorithms-models-should-you-choose-for-sentiment-analysis-2/

nb = MultinomialNB()

nb.fit(X_train_dtm, y_train)

metrics.accuracy_score(y_test, nb.predict(X_test_dtm))

nb_params = {
    'alpha': [1000,500,100,50,10,7,6,5,4,2,1,0.5,0.1,0.05,0.01,0.005,0.001]
}

clf_nb = GridSearchCV(nb, nb_params, verbose=5, n_jobs=-1)
clf_nb.fit(X_train_dtm, y_train)

clf_nb.best_params_

nb = MultinomialNB(alpha=0.5)

nb.fit(X_train_dtm, y_train)

metrics.accuracy_score(y_test, nb.predict(X_test_dtm))

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, nb.predict(X_test_dtm)))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, nb.predict(X_test_dtm), digits=3, target_names=target_names))

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

classes = np.unique(y_test)
fig, ax = plt.subplots()
cm = metrics.confusion_matrix(y_test, nb.predict(X_test_dtm), labels=classes)
sns.heatmap(cm, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False)
ax.set(xlabel="Pred", ylabel="True", title="Confusion matrix")
ax.set_yticklabels(labels=classes, rotation=0)
plt.show()

# roc curve and auc

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
    
probs = nb.predict_proba(X_test_dtm)
probs = probs[:, 1]
auc = roc_auc_score(y_test, probs)
print('AUC: %.2f' % auc)
fpr, tpr, thresholds = roc_curve(y_test, probs)
plot_roc_curve(fpr, tpr)

### Electra Model
Utilising pre-trained model using transformer model to learn the language

# import os
# import pickle

# import numpy as np
# import pandas as pd
# import tensorflow as tf
# import tensorflow.keras.layers as L
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import ModelCheckpoint
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, average_precision_score, roc_auc_score
# import matplotlib.pyplot as plt
# import transformers
# from transformers import AutoTokenizer, TFAutoModel, TFElectraModel, ElectraTokenizer
# from tqdm.notebook import tqdm
# from tokenizers import BertWordPieceTokenizer

# # Import Electra Model
# import pickle

# file = open("inputs/electra_model.pickle", "rb")
# electra_model = pickle.load(file)

# file = open("inputs/electra_tokenizer.pickle", "rb")
# electra_tokenizer = pickle.load(file)

# sentences = ["there is a shortage of capital, and we need extra financing", 
#              "growth is strong and we have plenty of liquidity", 
#              "there are doubts about our finances", 
#              "profits are flat"]

# inputs = electra_tokenizer(sentences, return_tensors="pt", padding=True)
# outputs = electra_model(**inputs)[0]

# labels = {0:'neutral', 1:'positive',2:'negative'}
# for idx, sent in enumerate(sentences):
#     print(sent, '----', labels[np.argmax(outputs.detach().numpy()[idx])])

    

### CNN Model
Reference: https://towardsdatascience.com/nlp-with-cnns-a6aa743bdc1e

# from keras.layers.convolutional import Conv1D
# from keras.layers import GlobalMaxPooling1D, Dropout, Activation

# # parameters
# batch_size = 32
# embedding_dims = 300
# filters = 250 # Length of the token vectors
# kernel_size = 3 # a window size of 3 tokens
# hidden_dims = 250 # number of neurons at the normal feedforward NN
# epochs = 2 

# model = Sequential()
# model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1, input_shape=(X_train_dtm.shape[1], embedding_dims)))
# # model.add(GlobalMaxPooling1D()) # default = 2
# # model.add(Dense(hidden_dims))
# # model.add(Dropout(0.2))
# # model.add(Activation('relu'))
# # model.add(Dense(1))
# # model.add(Activation('sigmoid'))
# model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
# model.fit(X_train_dtm,y_train,batch_size = batch_size,epochs = epochs , validation_data = (X_test,y_test))

### LSTM Neural Networks

# model = Sequential()
# model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.compile(loss='mse', optimizer='adam', metrics=['mse'])
# model.fit(X_train_dtm, y_train, epochs=3)

# # The maximum number of words to be used. (most frequent)
# MAX_NB_WORDS = 50000
# # Max number of words in each complaint.
# MAX_SEQUENCE_LENGTH = 250
# # This is fixed.
# EMBEDDING_DIM = 128

# model = Sequential()
# model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=28))
# model.add(SpatialDropout1D(0.2))
# model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(2, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# epochs = 5
# batch_size = 64

# history = model.fit(X_train_dtm, y_train, epochs=epochs, batch_size=batch_size)

### FinBERT Model

# # call the model
# from transformers import BertTokenizer, BertForSequenceClassification
# import pickle
# import numpy as np

# # load tokenizer
# file_dir = 'inputs/finbert_tokenizer.pickle'
# file = open(f"{file_dir}", "rb")
# tokenizer = pickle.load(file)

# # Load model
# file_dir = 'models/sentiment/'
# finbert = BertForSequenceClassification.from_pretrained(
#     f"{file_dir}", num_labels=3, local_files_only=True
# )

# sentences = ["there is a shortage of capital, and we need extra financing", 
#              "growth is strong and we have plenty of liquidity", 
#              "there are doubts about our finances", 
#              "profits are flat",
#              "very happy excited economy"]

# inputs = tokenizer(sentences, return_tensors="pt", padding=True)
# outputs = finbert(**inputs)[0]

# labels = {0:'neutral', 1:'positive',2:'negative'}
# for idx, sent in enumerate(sentences):
#     print(sent, '----', labels[np.argmax(outputs.detach().numpy()[idx])])

# # call the model
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
# import pickle
# import numpy as np

# # load tokenizer
# file_dir = 'inputs/prosus_finbert_tokenizer.pickle'
# file = open(f"{file_dir}", "rb")
# tokenizer = pickle.load(file)

# # Load model
# file_dir = 'models/sentiment/'
# finbert = BertForSequenceClassification.from_pretrained(
#     f"{file_dir}", num_labels = 3)

# sentences = ["there is a shortage of capital, and we need extra financing", 
#              "growth is strong and we have plenty of liquidity", 
#              "there are doubts about our finances", 
#              "profits are flat",
#             "extremely positive economy outlook"]

# inputs = tokenizer(sentences, return_tensors="pt", padding=True)
# outputs = finbert(**inputs)[0]

# labels = {0:'neutral', 1:'positive',2:'negative'}
# for idx, sent in enumerate(sentences):
#     print(sent, '----', labels[np.argmax(outputs.detach().numpy()[idx])])
    

# # prediction method for finbert model

# labels = {0: 0, 1: 1, 2: -1}

# def predict_fin(sentence):
#     """
#     Using finbert model to predict the financial sentiment
#     """
    
#     inputs = tokenizer(sentences, return_tensors="pt", padding=True)
#     outputs = finbert(**inputs)[0]
    
#     outcome = labels[np.argmax(outputs.detach().numpy()[idx])]
    
#     return outcome

# # predict df of the news article

# df['Predict'] = df['cleaned_headline'].apply(lambda x: predict_fin(x))
# df

### Ensemble Model

base_models = [
    ('xg', xg),
    ('svm', sv),
    ('lr', lr),
    ('knn', knn),
    ('nb', nb)
]

meta_model = svm.SVC(C=100, gamma=1, probability=True)

%%time
from sklearn.ensemble import StackingClassifier

stacked_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, n_jobs=-1, verbose=5)
stacked_model.fit(X_train_dtm, y_train)
stacked_model.score(X_test_dtm, y_test)

%%time

stacked_model_svm = StackingClassifier(estimators=base_models, final_estimator=lr, n_jobs=-1, verbose=5)
stacked_model_svm.fit(X_train_dtm, y_train)
stacked_model_svm.score(X_test_dtm, y_test)

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, stacked_model_svm.predict(X_test_dtm)))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, stacked_model_svm.predict(X_test_dtm), digits=3, target_names=target_names))

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

classes = np.unique(y_test)
fig, ax = plt.subplots()
cm = metrics.confusion_matrix(y_test, stacked_model_svm.predict(X_test_dtm), labels=classes)
sns.heatmap(cm, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False)
ax.set(xlabel="Pred", ylabel="True", title="Confusion matrix")
ax.set_yticklabels(labels=classes, rotation=0)
plt.show()

# roc curve and auc

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
    
probs = stacked_model_svm.predict_proba(X_test_dtm)
probs = probs[:, 1]
auc = roc_auc_score(y_test, probs)
print('AUC: %.2f' % auc)
fpr, tpr, thresholds = roc_curve(y_test, probs)
plot_roc_curve(fpr, tpr)

### FinBERT Model

# load in finbert model
import pickle

f = open('outputs/finbert.pickle', "rb")
finbert = pickle.load(f)

f = open('outputs/tokenizer.pickle', "rb")
tokenizer = pickle.load(f)

import numpy as np

# {0: 'neutral', 1: 'positive', 2: 'negative'}
labels = {0: 0, 1: 1, 2: 0}

def label_sentiment(sentence):  # take in list sentence as input
    """
    Transfer learning using FinBERT to train the model
    Note: sentences cannot be an empty list, else function will not work
    """
    assert len(sentence) != 0, "Sentences is an empty list, please debug to ensure that empty sentences are removed in df"

    inputs = tokenizer(sentence, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]
    
    val = labels[np.argmax(outputs.detach().numpy())]
    
    return val

import copy

fin_df = df.copy()

# Long computation (slower than other models) --> However, when productionalising, it is much faster.

fin_df['fin_label'] = fin_df['cleaned_headline'].apply(lambda x: label_sentiment(x)) # prediction method for finbert

from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, f1_score

print(f"Accuracy: {accuracy_score(fin_df['class_label'], fin_df['fin_label'])}")
print(f"Precision: {precision_score(fin_df['class_label'], fin_df['fin_label'])}")
print(f"ROC AUC: {roc_auc_score(fin_df['class_label'], fin_df['fin_label'])}")
print(f"F1 Score: {f1_score(fin_df['class_label'], fin_df['fin_label'])}")

### Model Evaluation
Summary of all the models tested

def model_performance(models, model_names):
    #create empty df with col names
    df = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 score', 'ROC AUC'])

    for n, model in enumerate(models):
        
        if model_names[n] == 'finbert':
            name = model_names[n]

            acc = metrics.accuracy_score(fin_df['class_label'], fin_df['fin_label'])
            prec = metrics.precision_score(fin_df['class_label'], fin_df['fin_label'])
            recall = metrics.recall_score(fin_df['class_label'], fin_df['fin_label'])
            f1 = metrics.f1_score(fin_df['class_label'], fin_df['fin_label'])
            roc_auc = metrics.roc_auc_score(fin_df['class_label'], fin_df['fin_label'])
            
        else:
        
            y_pred = model.predict(X_test_dtm)

            name = model_names[n]

            acc = metrics.accuracy_score(y_test, y_pred)
            prec = metrics.precision_score(y_test, y_pred)
            recall = metrics.recall_score(y_test, y_pred)
            f1 = metrics.f1_score(y_test, y_pred)
            roc_auc = metrics.roc_auc_score(y_test, y_pred)

        #append row to df
        df = df.append(
            {
                'Model' : name,
                'Accuracy': acc,
                'Precision': prec,
                'Recall': recall,
                'F1 score': f1,
                'ROC AUC': roc_auc
            }, ignore_index = True)

    return df.set_index('Model').transpose()

model_performance([finbert, xg, sv, lr, knn, nb, stacked_model], ['finbert', 'xg', 'svm', 'lr', 'knn', 'nb', 'stack'])

### Finalised Model

# # save model to pickle
# import pickle

# # save the model to disk
# filename = 'outputs/finalized_model_tuned.pickle'
# dbfile =  open(filename, 'wb')
# pickle.dump(stacked_model, dbfile)
# dbfile.close()

# # save the model to disk
# filename = 'outputs/tfidf_vect_tuned.pickle'
# dbfile =  open(filename, 'wb')
# pickle.dump(tfidf_vect, dbfile)
# dbfile.close()